In [1]:
import os

import pandas as pd
import numpy as np

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy

# these are for test purposes
import json
from pprint import pprint

app = Flask(__name__)

In [2]:
#################################################
# Database Setup
#################################################

app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///../db/NBATOP10STATS_v3.sqlite"
db = SQLAlchemy(app)
#
# # reflect an existing database into a new model
Base = automap_base()
# # reflect the tables
Base.prepare(db.engine, reflect=True)
#
# # Save references to each table
# Samples_Metadata = Base.classes.sample_metadata
# Samples = Base.classes.samples
NBATOP = Base.classes.nbatop

stmt = db.session.query(NBATOP).statement
df = pd.read_sql_query(stmt, db.session.bind)

D:\Anaconda3\lib\site-packages\flask_sqlalchemy\__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [3]:
df.head()

,ROWID,Player,Year,Team,G,Min,FGM,FGA,FG%,3PTM,...,FT%,Off,Def,Tot,Ast,TO,Stl,Blk,PF,Pts
0,1,Lebron James,2003,CLE,79,39.5,7.9,18.9,0.417,0.8,...,0.754,1.3,4.2,5.5,5.9,3.5,1.6,0.7,1.9,20.9
1,2,Lebron James,2004,CLE,80,42.4,9.9,21.1,0.472,1.4,...,0.750,1.4,6.0,7.4,7.2,3.3,2.2,0.7,1.8,27.2
2,3,Lebron James,2005,CLE,79,42.5,11.1,23.1,0.480,1.6,...,0.738,0.9,6.1,7.0,6.6,3.3,1.6,0.8,2.3,31.4
3,4,Lebron James,2006,CLE,78,41.0,9.9,20.8,0.476,1.3,...,0.698,1.1,5.7,6.7,6.0,3.2,1.6,0.7,2.2,27.3
4,5,Lebron James,2007,CLE,75,40.4,10.6,21.9,0.484,1.5,...,0.712,1.8,6.1,7.9,7.2,3.4,1.8,1.1,2.2,30.0


In [4]:
df = df.rename(columns={"FG%": "FGP", "3PTM": "TPTM", "3PTA": "TPTA", "3PT%": "TPTP", "FT%": "FTP"})

In [5]:
df.columns

Index(['ROWID', 'Player', 'Year', 'Team', 'G', 'Min', 'FGM', 'FGA', 'FGP',
       'TPTM', 'TPTA', 'TPTP', 'FTM', 'FTA', 'FTP', 'Off', 'Def', 'Tot', 'Ast',
       'TO', 'Stl', 'Blk', 'PF', 'Pts'],
      dtype='object')

In [6]:
list(df.columns)[2:]

['Year',
 'Team',
 'G',
 'Min',
 'FGM',
 'FGA',
 'FGP',
 'TPTM',
 'TPTA',
 'TPTP',
 'FTM',
 'FTA',
 'FTP',
 'Off',
 'Def',
 'Tot',
 'Ast',
 'TO',
 'Stl',
 'Blk',
 'PF',
 'Pts']

In [7]:
list(df.Player.unique())

['Lebron James',
 'Stephen Curry',
 'Kevin Durant',
 'Russel Westbrook',
 'Kyrie Irving',
 'James Harden',
 'Anthony Davis',
 'Kawhi Leonard',
 'Giannis Antetokounmpo',
 'Chris Paul']

In [8]:
#return all of lebrons stuff only
sample_data = df.loc[df['Player'] == 'Lebron James', ['Player', 'Year', 'Team', 'G', 'Min', 'FGM', 'FGA', 'FGP',
       'TPTM', 'TPTA', 'TPTP', 'FTM', 'FTA', 'FTP', 'Off', 'Def', 'Tot', 'Ast',
       'TO', 'Stl', 'Blk', 'PF', 'Pts']]
sample_data

,Player,Year,Team,G,Min,FGM,FGA,FGP,TPTM,TPTA,...,FTP,Off,Def,Tot,Ast,TO,Stl,Blk,PF,Pts
0,Lebron James,2003,CLE,79,39.5,7.9,18.9,0.417,0.8,2.7,...,0.754,1.3,4.2,5.5,5.9,3.5,1.6,0.7,1.9,20.9
1,Lebron James,2004,CLE,80,42.4,9.9,21.1,0.472,1.4,3.9,...,0.750,1.4,6.0,7.4,7.2,3.3,2.2,0.7,1.8,27.2
2,Lebron James,2005,CLE,79,42.5,11.1,23.1,0.480,1.6,4.8,...,0.738,0.9,6.1,7.0,6.6,3.3,1.6,0.8,2.3,31.4
3,Lebron James,2006,CLE,78,41.0,9.9,20.8,0.476,1.3,4.0,...,0.698,1.1,5.7,6.7,6.0,3.2,1.6,0.7,2.2,27.3
4,Lebron James,2007,CLE,75,40.4,10.6,21.9,0.484,1.5,4.8,...,0.712,1.8,6.1,7.9,7.2,3.4,1.8,1.1,2.2,30.0
5,Lebron James,2008,CLE,81,37.7,9.7,19.9,0.489,1.6,4.7,...,0.780,1.3,6.3,7.6,7.2,3.0,1.7,1.1,1.7,28.4
6,Lebron James,2009,CLE,76,39.1,10.1,20.1,0.503,1.7,5.1,...,0.767,0.9,6.4,7.3,8.6,3.4,1.6,1.0,1.6,29.7
7,Lebron James,2010,MIA,79,38.8,9.6,18.8,0.510,1.2,3.5,...,0.759,1.0,6.5,7.5,7.0,3.6,1.6,0.6,2.1,26.7
8,Lebron James,2011,MIA,62,37.5,10.0,18.9,0.531,0.9,2.4,...,0.771,1.5,6.4,7.9,6.2,3.4,1.9,0.8,1.5,27.1
9,Lebron James,2012,MIA,76,37.9,10.1,17.8,0.565,1.4,3.3,...,0.753,1.3,6.8,8.0,7.3,3.0,1.7,0.9,1.4,26.8


In [9]:
#convert to lists
data = {
    "year": sample_data.Year.values.tolist(),
    "team": sample_data.Team.values.tolist(),
    "games": sample_data.G.values.tolist(),
    "minutes": sample_data.Min.values.tolist(),
    "field_goals_made": sample_data.FGM.values.tolist(),
    "field_goals_attempt": sample_data.FGA.values.tolist(),
    "field_goals_percent": sample_data.FGP.values.tolist(),
    "three_points_made": sample_data.TPTM.values.tolist(),
    "there_points_attempt": sample_data.TPTA.values.tolist(),
    "three_points_percent": sample_data.TPTP.values.tolist(),
    "free_throws_made": sample_data.FTM.values.tolist(),
    "free_throws_attempt": sample_data.FTA.values.tolist(),
    "free_throws_percent": sample_data.FTP.values.tolist(),
    "off":  sample_data.Off.values.tolist(),
    "def":  sample_data.Def.values.tolist(),
    "tot":  sample_data.Tot.values.tolist(),
    "ast":  sample_data.Ast.values.tolist(),
    "turnovers":  sample_data.TO.values.tolist(),
    "steals":  sample_data.Stl.values.tolist(),
    "blocks":  sample_data.Blk.values.tolist(),
    "personal_fouls":  sample_data.PF.values.tolist(),
    "pts":  sample_data.Pts.values.tolist()
}

data

{'ast': [5.9,
  7.2,
  6.6,
  6.0,
  7.2,
  7.2,
  8.6,
  7.0,
  6.2,
  7.3,
  6.4,
  7.4,
  6.8,
  8.7,
  9.1],
 'blocks': [0.7,
  0.7,
  0.8,
  0.7,
  1.1,
  1.1,
  1.0,
  0.6,
  0.8,
  0.9,
  0.3,
  0.7,
  0.6,
  0.6,
  0.9],
 'def': [4.2,
  6.0,
  6.1,
  5.7,
  6.1,
  6.3,
  6.4,
  6.5,
  6.4,
  6.8,
  5.9,
  5.3,
  6.0,
  7.3,
  7.5],
 'field_goals_attempt': [18.9,
  21.1,
  23.1,
  20.8,
  21.9,
  19.9,
  20.1,
  18.8,
  18.9,
  17.8,
  17.6,
  18.5,
  18.6,
  18.2,
  19.3],
 'field_goals_made': [7.9,
  9.9,
  11.1,
  9.9,
  10.6,
  9.7,
  10.1,
  9.6,
  10.0,
  10.1,
  10.0,
  9.0,
  9.7,
  9.9,
  10.5],
 'field_goals_percent': [0.417,
  0.472,
  0.48,
  0.476,
  0.484,
  0.489,
  0.503,
  0.51,
  0.531,
  0.565,
  0.567,
  0.488,
  0.52,
  0.548,
  0.542],
 'free_throws_attempt': [5.8,
  8.0,
  10.3,
  9.0,
  10.3,
  9.4,
  10.2,
  8.4,
  8.1,
  7.0,
  7.6,
  7.7,
  6.5,
  7.2,
  6.5],
 'free_throws_made': [4.4,
  6.0,
  7.6,
  6.3,
  7.3,
  7.3,
  7.8,
  6.4,
  6.2,
  5.3,
  5

In [12]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
plotly.__version__

'3.2.1'

In [13]:
data['year']

[2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017]

In [14]:
#scatter plot
#https://plot.ly/python/line-and-scatter/

trace1 = go.Scatter(
    x = data['year'],
    y = data['free_throws_made']
)

plot_data = [trace1]

plot_layout = go.Layout(title="Free Throws Made by Lebron James per Year",
                  xaxis = {'title': 'Year'},
                  yaxis = {'title': 'Free Throws Made'})

plot_figure = go.Figure(data=plot_data, layout=plot_layout)

py.iplot(plot_figure)